In [1]:
import scanpy as sc
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import median_abs_deviation as mad
import numpy as np
import anndata as adt
from scipy.sparse import csr_matrix
import umap
import scanpy.external as sce

In [2]:
import warnings 
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [3]:
path = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/data.treatment_allconditions_pg_.xlsx'

# Dataset formating

In [4]:
sheets = pd.read_excel(path, sheet_name=None, header = None)

In [5]:
cles = list(sheets.keys())

In [6]:
cles

['Mock3dpi_pg',
 'GFP.NEG3dpi_pg',
 'GFP.POS3dpi_pg',
 'Mock6dpi_pg',
 'GFP.NEG6dpi_pg',
 'GFP.POS6dpi_pg']

In [7]:
adatas = []

for x in cles:
    sheet_part = sheets[x]
    var = sheet_part.iloc[1:, 0:5]  
    obs = pd.DataFrame(sheet_part.iloc[0, 5:])  
    data = sheet_part.iloc[1:, 5:].T

    adata = sc.AnnData(X=data, obs=obs, var=var) #create anndata
    
    #remodele the indexs
    #idx_var = pd.DataFrame(sheet_part.iloc[1:, 3]) 
    adata.obs.index = obs
    
    adata.obs.index = adata.obs.index.map(str)
    adata.obs.index = adata.obs.index.str.replace(r"[()']", "", regex=True)
    adata.obs.index = adata.obs.index.str.replace(";", "")
    adata.obs.index = adata.obs.index.str.replace(",", "")


    idx_var = pd.DataFrame(sheet_part.iloc[1:, 2])  # Index pour var (colonne cible)
    adata.var.index = [str(idx[0]) if isinstance(idx, tuple) else str(idx) for idx in idx_var.iloc[:, 0]]
    adata.var.index = [idx.replace("(", "").replace(")", "").replace("'", "").replace(";", "").replace(",", "") for idx in adata.var.index]
    
    adata.var.columns = sheet_part.iloc[0, 0:5]

    
    adata.obs = adata.obs.drop(columns=[0])
    
    adatas.append(adata)

In [8]:
adatas[0].var

,Protein.Group,Protein.Ids,Protein.Names,Genes,First.Protein.Description
CC187_HUMAN,A0A096LP49,A0A096LP49,CC187_HUMAN,CCDC187,Coiled-coil domain-containing protein 187
TVAL3_HUMAN,A0A0B4J271,A0A0B4J271,TVAL3_HUMAN,TRAV12-3,T cell receptor alpha variable 12-3
GAL3A_HUMANGAL3B_HUMAN,A0A0B4J2D5;P0DPI2,A0A0B4J2D5;P0DPI2,GAL3A_HUMAN;GAL3B_HUMAN,GATD3;GATD3B,Putative glutamine amidotransferase-like class...
PIOS1_HUMAN,A0A0B4J2F0,A0A0B4J2F0,PIOS1_HUMAN,PIGBOS1,Protein PIGBOS1
A0A7S6HGT3_CVH22,A0A7S6HGT3,A0A7S6HGT3,A0A7S6HGT3_CVH22,N,Nucleoprotein
...,...,...,...,...,...
F169A_HUMAN,Q9Y6X4,Q9Y6X4,F169A_HUMAN,FAM169A,Soluble lamin-associated protein of 75 kDa
ENPP4_HUMAN,Q9Y6X5,Q9Y6X5,ENPP4_HUMAN,ENPP4,Bis(5'-adenosyl)-triphosphatase ENPP4
NS1BP_HUMAN,Q9Y6Y0,Q9Y6Y0,NS1BP_HUMAN,IVNS1ABP,Influenza virus NS1A-binding protein
S23IP_HUMAN,Q9Y6Y8,Q9Y6Y8,S23IP_HUMAN,SEC23IP,SEC23-interacting protein


In [12]:
def load_it(i, adata):
    adata.obs['Id'] = cles[i]
    return adata

In [13]:
adatas = [load_it(i, ad) for i, ad in enumerate(adatas)]

In [18]:
for ad in adatas: 
    
    if isinstance(ad.X, np.ndarray):
         ad.X = ad.X.astype(np.float32)

    if isinstance(ad.X, csr_matrix):
         ad.X = csr_matrix(ad.X, dtype=np.float32)
    # Convertir toutes les colonnes de obs et var en chaînes
    ad.obs = ad.obs.astype(str)
    ad.var = ad.var.astype(str)

    # Convertir les index d'obs et var en chaînes
    ad.obs.index = ad.obs.index.map(str)
    ad.var.index = ad.var.index.map(str)

In [19]:
print(np.nan_to_num(adatas[0].X, nan=0.0).min())
print(np.nan_to_num(adatas[0].X, nan=0.0).max())

0.0
49146000.0


In [20]:
for ad in adatas:
    unique_id = ad.obs['Id'][0]
    file_path = os.path.join('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/', f'adata_{unique_id}.h5ad')
    ad.write(file_path)
    print(f"Saved: {file_path}")

Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/adata_Mock3dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/adata_GFP.NEG3dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/adata_GFP.POS3dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/adata_Mock6dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/adata_GFP.NEG6dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data/adata_GFP.POS6dpi_pg.h5ad


# Correction errors

In [21]:
adatas = []
data_dir = 'C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Data'

for filename in os.listdir(data_dir):
    if filename.endswith('.h5ad'):
            adata = sc.read_h5ad(os.path.join(data_dir, filename))
            adata.var_names_make_unique()
            adatas.append(adata)

print(f"Nombre de fichiers h5ad chargés avec succès : {len(adatas)}")


Nombre de fichiers h5ad chargés avec succès : 6


In [22]:
for ad in adatas:
    nbmiss = np.sum(np.isnan(ad.X))/ np.sum(np.size(ad.X)) *100# ad.X contient la matrice de données
    print(f"Pourcentage of NaN values: {nbmiss}%")

Pourcentage of NaN values: 14.870642901629822%
Pourcentage of NaN values: 15.965015451316821%
Pourcentage of NaN values: 36.266736224191334%
Pourcentage of NaN values: 26.015800618993325%
Pourcentage of NaN values: 34.17766997925349%
Pourcentage of NaN values: 28.933675895151083%


In [23]:
all_incorrect_indices = set()

for ad in adatas:
    print(ad.obs['Id'][0])
    ad.var['Genes'] = ad.var['Genes'].astype(str)
    if 'Genes' not in ad.var.columns:
        print("The column doesn't exist in adata.var")
    else:
        # Create a mask for the datas with date name
        date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%m-%d %H:%M:%S', errors='coerce').notna()
        other_date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%d-%m %H:%M:%S', errors='coerce').notna()

    # same for NaN
        nan_mask = ad.var['Genes'].astype(str).str.lower() == 'nan'

    # Combine
        incorrect_entries = date_mask | nan_mask | other_date_mask

        incorrect_genes = ad.var.loc[incorrect_entries, 'Genes']

        print("Incorrect name in column 'Genes' :")
        print(incorrect_genes)
        print(f"\nNombre total d'entrées incorrectes : {incorrect_entries.sum()}")
        print("\nIncorrect distribution :")
        print(incorrect_genes.value_counts())

   
        print("\nIncorrect Index :")
        print(incorrect_genes.index.tolist())
        all_incorrect_indices.update(incorrect_genes.index)


print("\nTous les indices incorrects uniques :")
print(sorted(list(all_incorrect_indices)))
print(f"\nNombre total d'indices incorrects uniques : {len(all_incorrect_indices)}")

GFP.NEG3dpi_pg
Incorrect name in column 'Genes' :
SEPT6_HUMAN    2006-09-01 00:00:00
SEPT2_HUMAN    2002-09-01 00:00:00
SEPT7_HUMAN    2007-09-01 00:00:00
YJ005_HUMAN                    nan
YI025_HUMAN                    nan
SEPT8_HUMAN    2008-09-01 00:00:00
SEPT5_HUMAN    2005-09-01 00:00:00
SEP11_HUMAN    2011-09-01 00:00:00
SEP10_HUMAN    2010-09-01 00:00:00
SEPT9_HUMAN    2009-09-01 00:00:00
Name: Genes, dtype: object

Nombre total d'entrées incorrectes : 10

Incorrect distribution :
Genes
nan                    2
2006-09-01 00:00:00    1
2002-09-01 00:00:00    1
2007-09-01 00:00:00    1
2008-09-01 00:00:00    1
2005-09-01 00:00:00    1
2011-09-01 00:00:00    1
2010-09-01 00:00:00    1
2009-09-01 00:00:00    1
Name: count, dtype: int64

Incorrect Index :
['SEPT6_HUMAN', 'SEPT2_HUMAN', 'SEPT7_HUMAN', 'YJ005_HUMAN', 'YI025_HUMAN', 'SEPT8_HUMAN', 'SEPT5_HUMAN', 'SEP11_HUMAN', 'SEP10_HUMAN', 'SEPT9_HUMAN']
GFP.NEG6dpi_pg
Incorrect name in column 'Genes' :
SYMC_HUMAN    2001-03-01 00:0

In [24]:
wrong_gene_name = {
"IGA2_HUMAN":"IGA2",
"IGL1_HUMAN":"IGL1",
"SEP10_HUMAN":"SEPTIN10",
"SEP11_HUMAN":"SEPTIN11",
"SEPT2_HUMAN":"SEPTIN2",
"SEPT5_HUMAN":"SEPTIN5",
"SEPT6_HUMAN":"SEPTIN6",
"SEPT7_HUMAN":"SEPTIN7",
"SEPT8_HUMAN":"SEPTIN8",
"SEPT9_HUMAN":"SEPTIN9",
"SYMC_HUMAN":"MARS1",
"SYMM_HUMAN":"MARS2",
"YI025_HUMAN":"YI025",
"YJ005_HUMAN":"YJ005"
}

In [25]:
def corriger_noms_genes(adatas, corrections):
    for ad in adatas:

        ad.var['Genes'] = ad.var['Genes'].astype(str)
        

        ad.var['Genes'] = ad.var['Genes'].replace(corrections)
        
        date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%m-%d %H:%M:%S', errors='coerce').notna()
        
        mask_a_corriger = date_mask | (ad.var['Genes'].str.lower() == 'nan')
        ad.var.loc[mask_a_corriger, 'Genes'] = ad.var.index[mask_a_corriger].str.split('_').str[0]


corriger_noms_genes(adatas, wrong_gene_name)

In [26]:
all_incorrect_indices = set()

for ad in adatas:
    print(ad.obs['Id'][0])
    ad.var['Genes'] = ad.var['Genes'].astype(str)
    if 'Genes' not in ad.var.columns:
        print("The column doesn't exist in adata.var")
    else:
        # Create a mask for the datas with date name
        date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%m-%d %H:%M:%S', errors='coerce').notna()
        other_date_mask = pd.to_datetime(ad.var['Genes'], format='%Y-%d-%m %H:%M:%S', errors='coerce').notna()

    # same for NaN
        nan_mask = ad.var['Genes'].astype(str).str.lower() == 'nan'

    # Combine
        incorrect_entries = date_mask | nan_mask | other_date_mask

        incorrect_genes = ad.var.loc[incorrect_entries, 'Genes']

        print("Incorrect name in column 'Genes' :")
        print(incorrect_genes)
        print(f"\nNombre total d'entrées incorrectes : {incorrect_entries.sum()}")
        print("\nIncorrect distribution :")
        print(incorrect_genes.value_counts())

   
        print("\nIncorrect Index :")
        print(incorrect_genes.index.tolist())
        all_incorrect_indices.update(incorrect_genes.index)


print("\nTous les indices incorrects uniques :")
print(sorted(list(all_incorrect_indices)))
print(f"\nNombre total d'indices incorrects uniques : {len(all_incorrect_indices)}")

GFP.NEG3dpi_pg
Incorrect name in column 'Genes' :
Series([], Name: Genes, dtype: object)

Nombre total d'entrées incorrectes : 0

Incorrect distribution :
Series([], Name: count, dtype: int64)

Incorrect Index :
[]
GFP.NEG6dpi_pg
Incorrect name in column 'Genes' :
Series([], Name: Genes, dtype: object)

Nombre total d'entrées incorrectes : 0

Incorrect distribution :
Series([], Name: count, dtype: int64)

Incorrect Index :
[]
GFP.POS3dpi_pg
Incorrect name in column 'Genes' :
Series([], Name: Genes, dtype: object)

Nombre total d'entrées incorrectes : 0

Incorrect distribution :
Series([], Name: count, dtype: int64)

Incorrect Index :
[]
GFP.POS6dpi_pg
Incorrect name in column 'Genes' :
Series([], Name: Genes, dtype: object)

Nombre total d'entrées incorrectes : 0

Incorrect distribution :
Series([], Name: count, dtype: int64)

Incorrect Index :
[]
Mock3dpi_pg
Incorrect name in column 'Genes' :
Series([], Name: Genes, dtype: object)

Nombre total d'entrées incorrectes : 0

Incorrect dis

In [27]:
os.makedirs('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/scP_bgn', exist_ok = True)

In [28]:
for ad in adatas:
    unique_id = ad.obs['Id'][0]
    file_path = os.path.join('C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/scP_bgn', f'adata_{unique_id}.h5ad')
    ad.write(file_path)
    print(f"Saved: {file_path}")

Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/scP_bgn\adata_GFP.NEG3dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/scP_bgn\adata_GFP.NEG6dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/scP_bgn\adata_GFP.POS3dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/scP_bgn\adata_GFP.POS6dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/scP_bgn\adata_Mock3dpi_pg.h5ad
Saved: C:/Users/MDV/Documents/Pipeline folder Yohan/scP/Output/scP_bgn\adata_Mock6dpi_pg.h5ad
